In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importing the dataset
df = pd.read_csv('Social_Network_Ads.csv')
df.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19.0,19000.0,0
1,15810944,Male,35.0,20000.0,0
2,15668575,Female,26.0,43000.0,0
3,15603246,Female,27.0,57000.0,0
4,15804002,Male,19.0,76000.0,0


In [3]:
X=df.iloc[:,[2,3]].values
y=df.iloc[:,4].values

In [4]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X=sc.fit_transform(X)

In [6]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [7]:
# Confusion Matrix
def evaluation_metric(y_test,y_pred):
    from sklearn.metrics import confusion_matrix
    cm=confusion_matrix(y_test,y_pred)
    return cm

In [8]:
from matplotlib.colors import ListedColormap
def visualisation(X_set,y_set,clf):
    X1, X2 = np.meshgrid(np.arange(start = X_set[:, 0].min() - 1, stop = X_set[:, 0].max() + 1, step = 0.01),
                     np.arange(start = X_set[:, 1].min() - 1, stop = X_set[:, 1].max() + 1, step = 0.01))
    plt.contourf(X1, X2, clf.predict(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape),
             alpha = 0.75, cmap = ListedColormap(('red', 'green')))
    plt.xlim(X1.min(), X1.max())
    plt.ylim(X2.min(), X2.max())
    for i, j in enumerate(np.unique(y_set)):
         plt.scatter(X_set[y_set == j, 0], X_set[y_set == j, 1],
                c = ListedColormap(('red', 'green'))(i), label = j)
   
    plt.title('Classifier')
    plt.xlabel('Age')
    plt.ylabel('Estimated Salary')
    plt.legend()
    plt.show()

In [10]:
# Kernel SVM
from sklearn.svm import SVC
svm_clf=SVC(kernel='rbf',random_state=0)
svm_clf.fit(X_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=0,
    shrinking=True, tol=0.001, verbose=False)

In [11]:
y_pred=svm_clf.predict(X_test)

In [12]:
cm_svm=evaluation_metric(y_test,y_pred)
cm_svm

array([[64,  4],
       [ 3, 29]], dtype=int64)

In [18]:
#Applying the k fold cross validation
from sklearn.model_selection import cross_val_score
accuracies=cross_val_score(estimator=svm_clf,X=X_train,y=y_train,cv=10)*100
accuracies.mean()
accuracies.std()

6.80430514209002

In [34]:
#Applying grid search for finding the best model and the best parameters
from sklearn.model_selection import GridSearchCV
parameters=[{'C':[1,10,100,1000],'kernel':['linear']},
           {'C':[1,10,100,1000],'kernel':['rbf'],'gamma':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,1]}]

grid_search=GridSearchCV(estimator=svm_clf,param_grid=parameters,scoring='accuracy',cv=10,n_jobs=-1)
grid_search.fit(X_train,y_train)
best_acc=(grid_search.best_score_)*100
best_parameters=grid_search.best_params_
best_parameters


F:\Users\Avijit Sen\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'C': 1, 'gamma': 0.4, 'kernel': 'rbf'}